In [1]:
from tensorflow.keras.models import load_model
from functions import margin_loss_fun_factory,DataGenerator,PredictGenerator,\
                        l2_distance_np,top_k,loop_distance,MAP,generate_feature,depthwise_maker
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
import time
import tensorflow as tf
import numpy as np
from functools import partial

In [2]:
color = False
#FFA_size = 4

Load model

In [3]:
feature_model = load_model('Models/feature_model_DenseNet_Depthwise2.h5')

In [4]:
conv_weight = np.load('Models/conv_weight_Depthwise2.npy')

Because model is not defined with head model, so had to do "one-time" recreation of head model. If training is done with update create_model2, then do not need to have this step.

In [12]:
layers = tf.keras.layers
models = tf.keras.models
feature1 = layers.Input(shape=(conv_weight.shape[1],))
feature2 = layers.Input(shape=(conv_weight.shape[1],))
x1 = layers.Lambda(lambda x : x[0]*x[1])([feature1, feature2])
x2 = layers.Lambda(lambda x : tf.abs(x[0] - x[1]))([feature1, feature2])
x3 = layers.Lambda(lambda x : tf.square(x[0] - x[1]))([feature1, feature2])
x4 = layers.Lambda(lambda x : x[0] + x[1])([feature1, feature2])
x = layers.Concatenate()([x1, x2, x3, x4])
x = layers.Reshape((4, 1, conv_weight.shape[1]), name='reshape1')(x)
output = layers.DepthwiseConv2D(kernel_size=(4,1),use_bias=False)(x)
output = layers.Lambda(lambda x:tf.squeeze(tf.reduce_mean(x,3,keepdims=True),(1,2)))(output)
head_model = models.Model([feature1,feature2],output)

In [15]:
l=head_model.layers[-2]

In [20]:
l.set_weights([conv_weight[:,np.newaxis,:,np.newaxis]])

In [22]:
head_model.compile(loss='mse',optimizer='sgd')

In [ ]:
#feature_model = load_model('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5', \
#                            custom_objects={'loss': margin_loss_fun_factory(0.1,5)})

In [23]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [24]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform = create_transform(aug)    

In [26]:
feature_train = generate_feature(Ids_train,transform,1,color,feature_model)
feature_val = generate_feature(Ids_val,transform,1,color,feature_model)

In [28]:
feature_train = np.squeeze(feature_train)
feature_val = np.squeeze(feature_val)

In [65]:
distance = depthwise_maker(conv_weight,1,(1,2))

In [68]:
%%timeit
d1 = distance(feature_train,feature_val[0:1])

88 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
d1 = distance(feature_train,feature_val[0:1])

In [113]:
gen = FeaturesGenerator(feature_train,feature_val[0:1],139)

In [114]:
%%timeit
d2 = head_model.predict_generator(gen,workers=2,use_multiprocessing=True)

230 ms ± 2.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [115]:
d2 = head_model.predict_generator(gen,workers=2,use_multiprocessing=True)

In [116]:
np.max(np.abs(d1-d2.flatten()))

7.6293945e-06